In [ ]:
! pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#API to fetch dataset from Kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 169MB/s]
100% 80.9M/80.9M [00:00<00:00, 148MB/s]


In [ ]:
#extracting the zip file
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [ ]:
#loading the data from csv file to pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
#checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [ ]:
#printing the first 5 rows
twitter_data.head()

,target,id,date,flag,user,text,stemmed content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoothttp://twitpic.com/2y1zl-awww,that'..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upsetcan'tupdatfacebooktextit...mightcriresult...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichandivemanitimeball.managsave50%restgobound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,wholebodifeelitchilikefire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclassno,behavall.i'mmad.here?can'ts..."


In [ ]:
#naming the coloumns and reading the dataset again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [ ]:
#checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [ ]:
#printing the first 5 rows
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0
stemmed content,0


In [ ]:
#checking the distribustion of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


Convert target values from "4" to "1"

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
#checking the distribustion of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0-----> Negative tweet

1-----> Positive tweet

**Stemming**

Stemming is the process of reducing a word to its Root word

Example: actor,actress,acting=act

In [ ]:
port_stem= PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content=re.sub('^a-zA-Z','',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=''.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoothttp://twitpic.com/2y1zl-awww,that'..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upsetcan'tupdatfacebooktextit...mightcriresult...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@kenichandivemanitimeball.managsave50%restgobound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,wholebodifeelitchilikefire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@nationwideclassno,behavall.i'mmad.here?can'ts..."


In [ ]:
print(twitter_data['stemmed content'])

0          @switchfoothttp://twitpic.com/2y1zl-awww,that'...
1          upsetcan'tupdatfacebooktextit...mightcriresult...
2          @kenichandivemanitimeball.managsave50%restgobound
3                                 wholebodifeelitchilikefire
4          @nationwideclassno,behavall.i'mmad.here?can'ts...
                                 ...                        
1599995                            wokeup.schoolbestfeelever
1599996    thewdb.com-coolhearoldwaltinterviews!â«http:/...
1599997                          readimojomakeover?askdetail
1599998      happi38thbirthdaybooallltime!!!tupacamarushakur
1599999    happi#charitytuesday@thenspcc@sparkschar@speak...
Name: stemmed content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data ['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seperating the data and label
X=twitter_data['stemmed content'].values
Y=twitter_data['target'].values
print(X)


["@switchfoothttp://twitpic.com/2y1zl-awww,that'bummer.shouldagotdavidcarrthirddayit.;d"
 "upsetcan'tupdatfacebooktextit...mightcriresultschooltodayalso.blah!"
 '@kenichandivemanitimeball.managsave50%restgobound' ...
 'readimojomakeover?askdetail'
 'happi38thbirthdaybooallltime!!!tupacamarushakur'
 'happi#charitytuesday@thenspcc@sparkschar@speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into training data and testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X_train)
print("------------------------------------------------")
print(X_test)

['watchsawivdrinklilwine' "@hatermagazini'min!"
 'eventhoughfavouritdrinkthinkvodkacokewipemindtimethinkimgonnafindnewdrink'
 ... 'eagermondayafternoon'
 "hopeeveryonmothergreatday!can'twaithearguystoretomorrow!"
 'lovewakefolgers.badvoicdeeperhis.']
------------------------------------------------
['@mmangen-fine!muchtimechattwitter.hubbibacksummer&amp;tenddominfreetime.'
 'ahmayshoww/ruthkim&amp;geoffreysanhueza'
 '@ishataramaybbayareathangdammit!' ...
 '@destini41neverthelesshooray!4700memberwondersafetrip!' 'feelwell'
 '@supersandro!!!!thankyou!']


In [ ]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test= vectorizer.transform(X_test)

In [ ]:
print(X_train)
print("------------------------------------------------")
print(X_test)

  (0, 2210162)	1.0
  (1, 818976)	0.8035662606386804
  (1, 1323737)	0.5952153095839928
  (2, 561394)	1.0
  (3, 2003138)	0.8415170928546275
  (3, 887642)	0.5402304900998244
  (4, 2055729)	0.7071067811865475
  (4, 641535)	0.7071067811865475
  (5, 416494)	0.8582407249127161
  (5, 543651)	0.5132473654109639
  (6, 1043627)	0.51288057217021
  (6, 2340887)	0.45919555319174626
  (6, 2340959)	0.51288057217021
  (6, 1857399)	0.51288057217021
  (7, 262501)	0.5
  (7, 682445)	0.5
  (7, 383075)	0.5
  (7, 182564)	0.5
  (8, 1501723)	0.7243619133837753
  (8, 1678702)	0.6894199144490939
  (9, 883504)	0.21347404293801664
  (9, 2102774)	0.24498682648641823
  (9, 372116)	0.20561609740274106
  (9, 62071)	0.6527374312572698
  (9, 564824)	0.6527374312572698
  :	:
  (1279990, 2054604)	0.4458373283998016
  (1279991, 536595)	0.7071067811865475
  (1279991, 1155440)	0.7071067811865475
  (1279992, 2179478)	0.5773502691896257
  (1279992, 1968178)	0.5773502691896257
  (1279992, 2033834)	0.5773502691896257
  (1279993, 

Training the Machine Learning Model

Logistic Regression

In [ ]:
model= LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [ ]:
#accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.9268046875


In [ ]:
#accuracy score on the testing data
X_test_prediction=model.predict(X_test)
testing_data_accuracy= accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('Accuracy score on the testing data:',testing_data_accuracy)

Accuracy score on the testing data: 0.63105


Model accuracy= 63.1%

Saving the trained model

In [ ]:
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb')) #wb=writing to the file

Using the saved model for future predictions

In [ ]:
#loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb')) #reading the file in binary format

In [ ]:
X_new= X_test[200] #200th data from X_test
print(Y_test[200])

prediction=loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[0]
Negative Tweet


In [ ]:
X_new= X_test[3] #200th data from X_test
print(Y_test[3])

prediction=loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

0
[0]
Negative Tweet
